In [1]:
!pip install faker

In [2]:
import pandas
import faker
import sqlalchemy
import pyodbc # Este é o mais provável de precisar instalar
print("Todas as bibliotecas importadas com sucesso!")

Todas as bibliotecas importadas com sucesso!


In [2]:
# logistics_flow_simulator.py - Script para Recarregar APENAS HistoricoOperacoes

import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
import urllib # Importe urllib para codificar a string de conexão

# --- Configurações do Banco de Dados ---
DB_SERVER = 'BRUNO\\SQLEXPRESS'
DB_DATABASE = 'LoggiConnect'
DB_USERNAME = 'BRUNO\\Bruno Queles'
DB_PASSWORD = ''
DB_DRIVER = '{ODBC Driver 17 for SQL Server}'

# --- Inicializa o Faker (usado aqui apenas para funções internas de data/hora) ---
fake = Faker('pt_BR')

# --- Dados de Referência (usados na geração do histórico) ---
STATUS_IDS = {
    'Coleta Realizada': 10,
    'Em Trânsito': 11,
    'Objeto em Rota de Entrega': 12,
    'Entrega Realizada': 13,
    'Atraso na Entrega': 1,
    'Extravio de Encomenda': 2,
    'Avaria no Produto': 3,
    'Destinatário Ausente': 4,
    'Endereço Incorreto': 5,
    'Recusa da Mercadoria': 6,
    'Coleta Não Realizada': 7,
    'Roubo / Furto': 8,
    'Restrição de Entrega': 9,
    'Objeto Devolvido': 14,
    'Problema Fiscal': 15,
}

# Lista de Cidades por UF (simplificada para o exemplo)
CIDADES_POR_UF = {
    'SP': ['São Paulo', 'Campinas', 'Guarulhos', 'Ribeirão Preto', 'Santos'],
    'RJ': ['Rio de Janeiro', 'Niterói', 'Duque de Caxias', 'Nova Iguaçu', 'Campos dos Goytacazes'],
    'MG': ['Belo Horizonte', 'Uberlândia', 'Contagem', 'Juiz de Fora', 'Montes Claros'],
    'ES': ['Vitória', 'Vila Velha', 'Serra', 'Cariacica', 'Linhares']
}
UFS_SUDESTE = ['SP', 'RJ', 'MG', 'ES']

# --- Função para Gerar Histórico de Ocorrências para uma Operação ---
def generate_history_for_operation(operation_row):
    history = []
    id_operacao = operation_row['id_operacao']
    codigo_rastreio = operation_row['codigo_rastreio']
    data_pedido = operation_row['data_pedido']
    data_previsao_entrega = operation_row['data_previsao_entrega']
    data_entrega_real = operation_row['data_entrega_real'] 
    status_final_id = operation_row['status_entrega_id']
    origem_cidade = operation_row['origem_cidade']
    destino_cidade = operation_row['destino_cidade']
    origem_uf = operation_row['origem_uf']
    destino_uf = operation_row['destino_uf']

    status_flow = [
        (STATUS_IDS['Coleta Realizada'], 'Coleta do produto realizada com sucesso.'),
        (STATUS_IDS['Em Trânsito'], 'Objeto em trânsito entre unidades.'),
        (STATUS_IDS['Objeto em Rota de Entrega'], 'Objeto saiu para entrega final.'),
        (STATUS_IDS['Entrega Realizada'], 'Entrega finalizada com sucesso.')
    ]

    local_eventos = {
        STATUS_IDS['Coleta Realizada']: origem_cidade + ' / ' + origem_uf,
        STATUS_IDS['Entrega Realizada']: destino_cidade + ' / ' + destino_uf,
    }

    if isinstance(data_pedido, datetime):
        current_history_datetime = data_pedido
    elif isinstance(data_pedido, pd.Timestamp):
        current_history_datetime = data_pedido.to_pydatetime()
    else:
        current_history_datetime = datetime.combine(data_pedido, datetime.min.time())
    
    history.append({
        'id_operacao': id_operacao,
        'codigo_rastreio': codigo_rastreio,
        'id_tipo_ocorrencia': STATUS_IDS['Coleta Realizada'],
        'data_hora_evento': current_history_datetime,
        'local_evento': local_eventos[STATUS_IDS['Coleta Realizada']],
        'observacoes': 'Coleta do produto realizada com sucesso.'
    })

    if status_final_id == STATUS_IDS['Entrega Realizada']:
        for status_id, obs_text in status_flow[1:-1]:
            days_passed = random.randint(1, 3)
            proposed_datetime = current_history_datetime + timedelta(days=days_passed, hours=random.randint(1, 23), minutes=random.randint(1, 59))
            
            if data_entrega_real:
                real_delivery_datetime = datetime.combine(data_entrega_real, datetime.max.time())
                if proposed_datetime > real_delivery_datetime:
                    proposed_datetime = real_delivery_datetime - timedelta(hours=random.randint(1,48), minutes=random.randint(1,59))
                    if proposed_datetime < current_history_datetime:
                        proposed_datetime = current_history_datetime + timedelta(minutes=random.randint(5,60))
            
            current_history_datetime = proposed_datetime

            if status_id == STATUS_IDS['Em Trânsito']:
                intermediate_city = random.choice(CIDADES_POR_UF[random.choice(UFS_SUDESTE)])
                local_evento_intermediate = intermediate_city + ' / ' + random.choice(UFS_SUDESTE)
            elif status_id == STATUS_IDS['Objeto em Rota de Entrega']:
                local_evento_intermediate = destino_cidade + ' / ' + destino_uf
            else:
                local_evento_intermediate = fake.city() + ' / ' + fake.state_abbr()

            history.append({
                'id_operacao': id_operacao,
                'codigo_rastreio': codigo_rastreio,
                'id_tipo_ocorrencia': status_id,
                'data_hora_evento': current_history_datetime,
                'local_evento': local_evento_intermediate,
                'observacoes': obs_text
            })

        final_delivery_datetime = datetime.combine(data_entrega_real, fake.time_object()) if data_entrega_real else current_history_datetime
        if final_delivery_datetime < current_history_datetime:
            final_delivery_datetime = current_history_datetime + timedelta(minutes=random.randint(5, 60))
        if final_delivery_datetime.date() < data_pedido.date():
            final_delivery_datetime = datetime.combine(data_pedido.date(), datetime.min.time()) + timedelta(days=random.randint(1,3), hours=random.randint(1,23), minutes=random.randint(1,59))
            if final_delivery_datetime < current_history_datetime:
                final_delivery_datetime = current_history_datetime + timedelta(minutes=random.randint(5, 60))


        history.append({
            'id_operacao': id_operacao,
            'codigo_rastreio': codigo_rastreio,
            'id_tipo_ocorrencia': status_final_id,
            'data_hora_evento': final_delivery_datetime,
            'local_evento': local_eventos[STATUS_IDS['Entrega Realizada']],
            'observacoes': 'Entrega finalizada com sucesso.'
        })
    else:
        problem_occurrence_datetime = fake.date_time_between(
            start_date=current_history_datetime,
            end_date=datetime.combine(data_previsao_entrega, datetime.max.time()) + timedelta(days=random.randint(1, 10)),
            tzinfo=None
        )
        if problem_occurrence_datetime < current_history_datetime:
            problem_occurrence_datetime = current_history_datetime + timedelta(minutes=random.randint(5,60))
            
        if status_final_id == STATUS_IDS['Atraso na Entrega']:
            obs = 'Entrega atrasada devido a imprevistos operacionais.'
            loc = fake.city() + ' / ' + fake.state_abbr()
        elif status_final_id == STATUS_IDS['Destinatário Ausente']:
            obs = 'Destinatário ausente na tentativa de entrega.'
            loc = destino_cidade + ' / ' + destino_uf
        elif status_final_id == STATUS_IDS['Problema Fiscal']:
            obs = 'Problema fiscal detectado na mercadoria. Retida em fiscalização.'
            loc = fake.city() + ' / ' + fake.state_abbr()
        else:
            obs = fake.text(max_nb_chars=100)
            loc = fake.city() + ' / ' + fake.state_abbr()

        history.append({
            'id_operacao': id_operacao,
            'codigo_rastreio': codigo_rastreio,
            'id_tipo_ocorrencia': status_final_id,
            'data_hora_evento': problem_occurrence_datetime,
            'local_evento': loc,
            'observacoes': obs
        })
    
    history.sort(key=lambda x: x['data_hora_evento'])
    
    return history


# --- Função para Obter Conexão com o Banco de Dados ---
def get_db_connection():
    params = urllib.parse.quote_plus(
        f"DRIVER={DB_DRIVER};"
        f"SERVER={DB_SERVER};"
        f"DATABASE={DB_DATABASE};"
        f"Trusted_Connection=yes;"
    )
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    engine = create_engine(conn_str)
    return engine

# --- Bloco Principal: Simulação de Carga (Apenas Histórico) ---
if __name__ == "__main__":
    engine = get_db_connection()

    print("\n--- Testando conexão com o SQL Server ---")
    try:
        with engine.connect() as connection:
            result = connection.execute(text("SELECT 1 AS ConnectionTest")).scalar()
            if result == 1:
                print("Conexão com o SQL Server estabelecida com sucesso!")
            else:
                print("Erro inesperado ao testar a conexão com o SQL Server.")
    except Exception as e:
        print(f"Erro ao conectar ao SQL Server: {e}")
        print("Verifique seu servidor, nome da instância, nome do banco de dados e driver ODBC.")
        exit()

    # --- 1. Extrair os dados necessários da Logistica.Operacoes (já populada e correta) ---
    print("\n--- Extraindo operações da Logistica.Operacoes para gerar o histórico ---")
    # Buscamos os dados completos da Operacoes, pois precisamos de todos os detalhes
    # para gerar um histórico realista (data_pedido, data_previsao_entrega, etc.)
    query_operations = """
    SELECT
        id_operacao,
        id_cliente,
        id_transportadora,
        data_pedido,
        data_previsao_entrega,
        data_entrega_real,
        origem_uf,
        origem_cidade,
        destino_uf,
        destino_cidade,
        peso_kg,
        volume_m3,
        valor_frete,
        status_entrega_id,
        codigo_rastreio
    FROM Logistica.Operacoes;
    """
    
    try:
        df_operations_for_history = pd.read_sql(query_operations, engine)
        print(f"Total de {len(df_operations_for_history)} operações encontradas em Logistica.Operacoes para gerar histórico.")
        if df_operations_for_history.empty:
            print("Nenhuma operação em Logistica.Operacoes para gerar histórico. Encerrando o script.")
            exit()
    except Exception as e:
        print(f"Erro ao extrair dados da Logistica.Operacoes para geração de histórico: {e}")
        exit()

    # --- 2. Geração e Carga do Histórico de Ocorrências (Logistica.HistoricoOperacoes) ---
    print("\n--- Gerando e inserindo histórico de ocorrências para as operações ---")
    all_history_data = []
    
    for index, row in df_operations_for_history.iterrows():
        all_history_data.extend(generate_history_for_operation(row))
    
    df_history = pd.DataFrame(all_history_data)
    
    # Remove a coluna 'id_historico' se ela for auto-incremento no seu DB
    if 'id_historico' in df_history.columns:
        df_history = df_history.drop(columns=['id_historico'])

    print(f"Total de {len(df_history)} registros de histórico gerados.")

    print(f"--- Inserindo {len(df_history)} dados na tabela Logistica.HistoricoOperacoes (append). ---")
    try:
        df_history.to_sql(
            name='HistoricoOperacoes',
            con=engine,
            schema='Logistica',
            if_exists='append',
            index=False,
            chunksize=500
        )
        print(f"Todos os {len(df_history)} dados foram inseridos com sucesso na tabela Logistica.HistoricoOperacoes!")

    except Exception as e:
        print(f"Erro ao gerar ou inserir histórico de ocorrências: {e}")
        print("Certifique-se de que Logistica.HistoricoOperacoes foi truncada antes desta carga e que o esquema da tabela está correto (incluindo 'codigo_rastreio').")


--- Testando conexão com o SQL Server ---
Conexão com o SQL Server estabelecida com sucesso!

--- Extraindo operações da Logistica.Operacoes para gerar o histórico ---
Total de 49371 operações encontradas em Logistica.Operacoes para gerar histórico.

--- Gerando e inserindo histórico de ocorrências para as operações ---
Total de 177002 registros de histórico gerados.
--- Inserindo 177002 dados na tabela Logistica.HistoricoOperacoes (append). ---
Todos os 177002 dados foram inseridos com sucesso na tabela Logistica.HistoricoOperacoes!
